In [1]:
import os
import numpy as np
import pandas as pd
from keras.models import load_model
from keras_bert import get_custom_objects
from utils import mysqlDatabase
from myBertTools import myBertModel, myTokenizer
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

Init Plugin
Init Graph Optimizer
Init Kernel


/var/folders/17/dsyzw2j96ggfgmrjcsnbtz700000gn/T/ipykernel_63867/65952826.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [19]:
def class_predict_fuc(text):
    text = str(text)[:100]
    x1, x2 = MyTokenizer.encode(first=text)
    x1, x2 = np.array([x1]), np.array([x2])
    outcome = model.predict([x1, x2])
    percentage = outcome.max()
    label = label_dic[outcome.argmax()]
    return label, percentage

In [20]:
def multi_label_predict(text):
    text = str(text)[:100]
    x1, x2 = MyTokenizer.encode(first=text)
    x1, x2 = np.array([x1]), np.array([x2])
    outcome = model.predict([x1, x2])
    multi_label_set = []
    label_idx = np.where(outcome >= 0.15)[1]
    for idx in label_idx:
        multi_label_set.append(label_dic[idx])
    return multi_label_set

In [12]:
if __name__ == '__main__':
    pretrained_path = '/Users/jackyfu/Desktop/hwf87_git/bert_wwm/'
    config_path = os.path.join(pretrained_path, 'bert_config.json')
    checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
    vocab_path = os.path.join(pretrained_path, 'vocab.txt')
    model_path = '/Users/jackyfu/Desktop/hwf87_git/Dcard_post_classification/model_output/dcard_post_cls_bert_2.h5'
    dic_path = '/Users/jackyfu/Desktop/hwf87_git/Dcard_post_classification/model_output/dcard_cate_label_dic_2.npy'
    ## Load Model
    MyBertModel = myBertModel(pretrained_path, config_path, checkpoint_path, vocab_path)
    token_dict = MyBertModel.get_token_dict()
    MyTokenizer = myTokenizer(token_dict)
    model = load_model(model_path, custom_objects=get_custom_objects())
    label_dic = np.load(dic_path, allow_pickle=True).item()
    ## predict
    text = '''關於80人以上遠距的信 真是疑問滿滿… 
    1.所以第一週遠距的課要怎麼加簽？
    2.是選課上限80以上的就確定第一週遠距，上限80以下第一週不遠距嗎？還是看選課人數？
    3.那如果看選課人數
    '''
    # label, confidance = class_predict_fuc(text)
    # print('看板預測: ', label, ' 信心水準: ', confidance)
    multi_label_set = multi_label_predict(text)
    print('predict multi-label set: ', multi_label_set)

2021-09-05 22:35:00.900680: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


predict multi-label set:  ['考試', '臺灣大學']


## Performance testing

In [21]:
import yaml
with open('config.yml', 'r') as stream:
        myconfig = yaml.load(stream, Loader=yaml.CLoader)
database_username = myconfig['mysql_database']['database_username']
database_password = myconfig['mysql_database']['database_password']
database_ip       = myconfig['mysql_database']['database_ip']
database_name     = myconfig['mysql_database']['database_name']

MysqlDatabase = mysqlDatabase(database_username, database_password, database_ip, database_name)

In [14]:
sql = '''
SELECT df.name forums_name, dp.*
FROM Bigdata.dcard_posts dp
left join Bigdata.dcard_forums df on dp.forumid = df.id
WHERE 1=1
'''
df = MysqlDatabase.select_table(sql)

Successfully select from Bigdata table


In [22]:
df['text'] = df['title'] + ' ' + df['excerpt'] + ' ' + df['topics']
df_test = df[df.createdAt >= '2021-09-05'][['forums_name', 'id', 'title', 'excerpt', 'topics', 'text']]
df_test['predict_forums_name'] = df_test['text'].progress_apply(multi_label_predict)

  0%|          | 0/491 [00:00<?, ?it/s]

In [27]:
df_test.sample(3)

,forums_name,id,title,excerpt,topics,text,predict_forums_name
17417,穿搭,236888918,今年夏天穿了什麼,今年因為疫情都不太出門，導致出門走走拍拍的就只有那幾天⋯,"['夏天', '日常穿搭', '女生穿搭']","今年夏天穿了什麼 今年因為疫情都不太出門，導致出門走走拍拍的就只有那幾天⋯ ['夏天', '...","[女生穿搭, 日常穿搭, 穿搭]"
16002,穿搭,236890146,童裝買賣,最近在經營童裝店鋪，歡迎有興趣的可以參觀參觀，下面都是實體拍照，自己驗證過的材質 真的超級舒...,"['童裝', '衣服', '日常穿搭', '蝦皮']",童裝買賣 最近在經營童裝店鋪，歡迎有興趣的可以參觀參觀，下面都是實體拍照，自己驗證過的材質 ...,"[日常穿搭, 穿搭]"
10531,美食,236890450,一個人的海底撈,好久沒吃撈撈了，以前都一群人開鍋比較便宜也可以吃比較多種，疫情的關係梅花坐 變成一人一桌，很...,['海底撈'],一個人的海底撈 好久沒吃撈撈了，以前都一群人開鍋比較便宜也可以吃比較多種，疫情的關係梅花坐 ...,"[美食, 食記]"
